## macrology 201

*"Macrology 201" workshop at flatMap 2014*: http://2014.flatmap.no/speakers/burmako.html

https://github.com/scalamacros/macrology201

jupyter-scala: not much comment in this notebook yet, see the macrology commit descriptions (linked along the way). Only steps 1 to 17 for now.

The only pitfalls from jupyter-scala are:
* `special.wrap.obj` needs to be imported in cells with macro implementations,
* `interpreter.init()` has to be called from time to time after a macro definition and before any call to it.

This could be automated in the future.

The first point is an "implementation" detail: this wraps these macro implementations in singletons rather than classes, which is required for them to be called as macros. About the second point, it can possibly be done in the same cell as the macro implementation (not tested), but in a different cell here than the call to the macro. It re-initializes the compiler instance. Not always needed though, for unknown reasons.

A definition that will be repeatedly used.

In [1]:
class C { override def toString = "C" }

defined class C

## Part 1

https://github.com/scalamacros/macrology201/commits/part1

### step 0: Initializing

https://github.com/scalamacros/macrology201/commit/05b496295e7ef9c7a41d7cabeaf326d819d54198

In [2]:
import special.wrap.obj
import scala.language.experimental.macros
import scala.reflect.macros.blackbox.Context

object Macros {
  def impl(c: Context): c.Tree = {
    import c.universe._
    q"""println("Hello World")"""
  }

  def hello: Unit = macro impl
}

import scala.language.experimental.macros
import scala.reflect.macros.blackbox.Context
defined object Macros

In [3]:
Macros.hello

Hello World


### step 1: definition of Optional[A] and signature for getOrElse 
### step 2: can't seem to make a non-macro implementation inlined 

https://github.com/scalamacros/macrology201/commit/e5ca9e5f716536856475057d6ed3ea6287720c12
https://github.com/scalamacros/macrology201/commit/69bb76b1360406c421e86bdeeba40b286d741649

In [4]:
import special.wrap.obj
import scala.language.experimental.macros
import scala.reflect.macros.blackbox.Context

// Allocation-free option type for Scala
// Inspired by https://github.com/arosenberger/nalloc

final class Optional[+A >: Null](val value: A) extends AnyVal {
  def get: A = value
  def isEmpty = value == null
  @inline final def getOrElse[B >: A](alt: => B): B = if (isEmpty) alt else value
  override def toString = if (isEmpty) "<empty>" else s"$value"
}

import scala.language.experimental.macros
import scala.reflect.macros.blackbox.Context
defined class Optional

In [5]:
val x = new Optional(new C)
println(x)
val y = new Optional(null)
println(y)

C
<empty>


x: specialObjCmd3.Optional[cmd4.INSTANCE.$ref$cmd0.C] = C
y: specialObjCmd3.Optional[Null] = <empty>

In [6]:
val x = new Optional(new C)
val x1 = x.getOrElse(new C)
println(x, x1)

val y = new Optional(null)
val y1 = y.getOrElse({ println("hello"); new C })
println(y, y1)

(C,C)
hello
(<empty>,C)


x: specialObjCmd3.Optional[cmd5.INSTANCE.$ref$cmd0.C] = C
x1: cmd5.INSTANCE.$ref$cmd0.C = C
y: specialObjCmd3.Optional[Null] = <empty>
y1: cmd5.INSTANCE.$ref$cmd0.C = C

### step 3: first attempt at writing the getOrElse macro, type error in expansion

https://github.com/scalamacros/macrology201/commit/ade3046228ea6afb4e1cf33c146f9ace87afd2cd

In [7]:
import special.wrap.obj
import scala.language.experimental.macros
import scala.reflect.macros.blackbox.Context

// Allocation-free option type for Scala
// Inspired by https://github.com/arosenberger/nalloc

final class Optional[+A >: Null](val value: A) extends AnyVal {
  def get: A = value
  def isEmpty = value == null
  // @inline final def getOrElse[B >: A](alt: => B): B = if (isEmpty) alt else value
  def getOrElse[B >: A](alt: => B): B = macro OptionalMacros.getOrElse
  override def toString = if (isEmpty) "<empty>" else s"$value"
}

class OptionalMacros(val c: Context) {
  def getOrElse(alt: c.Tree): c.Tree = {
    import c.universe._
    q"if (isEmpty) $alt else value"
  }
}

import scala.language.experimental.macros
import scala.reflect.macros.blackbox.Context
defined class Optional
defined class OptionalMacros

In [8]:
val x = new Optional(new C)
val x1 = x.getOrElse(new C)
println(x, x1)

val y = new Optional(null)
val y1 = y.getOrElse({ println("hello"); new C })
println(y, y1)

: 

The above error is expected.

### step 4: trying to debug the expansion failure using showCode, not much luck so far

https://github.com/scalamacros/macrology201/commit/c1d40e02ad74415a26c3888ea7f0f3d330e587ca

In [9]:
import special.wrap.obj
import scala.language.experimental.macros
import scala.reflect.macros.blackbox.Context

// Allocation-free option type for Scala
// Inspired by https://github.com/arosenberger/nalloc

final class Optional[+A >: Null](val value: A) extends AnyVal {
  def get: A = value
  def isEmpty = value == null
  // @inline final def getOrElse[B >: A](alt: => B): B = if (isEmpty) alt else value
  def getOrElse[B >: A](alt: => B): B = macro OptionalMacros.getOrElse
  override def toString = if (isEmpty) "<empty>" else s"$value"
}

class OptionalMacros(val c: Context) {
  def getOrElse(alt: c.Tree): c.Tree = {
    import c.universe._
    val result = q"if (isEmpty) $alt else value"
    println(showCode(result))
    result
  }
}

import scala.language.experimental.macros
import scala.reflect.macros.blackbox.Context
defined class Optional
defined class OptionalMacros

In [10]:
val x = new Optional(new C)
val x1 = x.getOrElse(new C)
println(x, x1)

val y = new Optional(null)
val y1 = y.getOrElse({ println("hello"); new C })
println(y, y1)

: 

The above error is expected.

### step 5: -Xprint:typer reveals the big picture, finally making the bug apparent

https://github.com/scalamacros/macrology201/commit/1353dbb761b0c5aaadce1b10c9ab1deb70e087c4

In [11]:
interpreter.init(Seq("-Xprint:typer"))

[[syntax trees at end of                     typer]] // Main.scala
package <empty> {
  object $dummy extends scala.AnyRef {
    def <init>(): $dummy.type = {
      $dummy.super.<init>();
      ()
    }
  }
}



In [12]:
import special.wrap.obj
import scala.language.experimental.macros
import scala.reflect.macros.blackbox.Context

// Allocation-free option type for Scala
// Inspired by https://github.com/arosenberger/nalloc

final class Optional[+A >: Null](val value: A) extends AnyVal {
  def get: A = value
  def isEmpty = value == null
  // @inline final def getOrElse[B >: A](alt: => B): B = if (isEmpty) alt else value
  def getOrElse[B >: A](alt: => B): B = macro OptionalMacros.getOrElse
  override def toString = if (isEmpty) "<empty>" else s"$value"
}

class OptionalMacros(val c: Context) {
  def getOrElse(alt: c.Tree): c.Tree = {
    import c.universe._
    q"if (isEmpty) $alt else value"
  }
}

[[syntax trees at end of                     typer]] // Main.scala
package <empty> {
  import ammonite.api.IvyConstructor.{ArtifactIdExt, GroupIdExt, ResolverNameExt};
  import scala.reflect.macros.blackbox.Context;
  import specialObjCmd7.OptionalMacros;
  import ReplBridge.shell.{load, publish, evidence, pprintConfig, interpreter};
  import scala.language.experimental.macros;
  object specialObjCmd9$Main extends scala.AnyRef {
    def <init>(): specialObjCmd9$Main.type = {
      specialObjCmd9$Main.super.<init>();
      ()
    };
    def $main(): Iterator[Iterator[String]] = {
      val $user: specialObjCmd9.type = specialObjCmd9;
      scala.`package`.Iterator.apply[Iterator[String]](scala.`package`.Iterator.apply[String](ReplBridge.shell.shellPrintImport("scala.language.experimental.macros")), scala.`package`.Iterator.apply[String](ReplBridge.shell.shellPrintImport("scala.reflect.macros.blackbox.Context")), scala.`package`.Iterator.apply[String](ReplBridge.shell.shellPrintDef("clas

import scala.language.experimental.macros
import scala.reflect.macros.blackbox.Context
defined class Optional
defined class OptionalMacros

In [13]:
val x = new Optional(new C)
val x1 = x.getOrElse(new C)
println(x, x1)

val y = new Optional(null)
val y1 = y.getOrElse({ println("hello"); new C })
println(y, y1)

[[syntax trees at end of                     typer]] // Main.scala
package <empty> {
  object cmd10$Main extends scala.AnyRef {
    def <init>(): cmd10$Main.type = {
      cmd10$Main.super.<init>();
      ()
    };
    private[this] val $ref$cmd0: cmd0.INSTANCE.$user.type = cmd0.INSTANCE.$user;
    <stable> <accessor> def $ref$cmd0: cmd0.INSTANCE.$user.type = cmd10$Main.this.$ref$cmd0;
    private[this] val $ref$cmd5: cmd5.INSTANCE.$user.type = cmd5.INSTANCE.$user;
    <stable> <accessor> def $ref$cmd5: cmd5.INSTANCE.$user.type = cmd10$Main.this.$ref$cmd5;
    import ammonite.api.IvyConstructor.{ArtifactIdExt, ResolverNameExt, GroupIdExt};
    import specialObjCmd9.Optional;
    import ReplBridge.shell.{load, publish, evidence, pprintConfig, interpreter};
    import scala.language.experimental.macros;
    import cmd10$Main.this.$ref$cmd0.C;
    import cmd10$Main.this.$ref$cmd5.{y1, y, x1, x};
    def $main(): Iterator[Iterator[String]] = {
      val $user: cmd10.INSTANCE.$user.type = c

: 

The above error is expected.

### step 6: but how do we get to the "this" of the macro application? 

https://github.com/scalamacros/macrology201/commit/8af23202542f6e512641df62ac98ff49614babf2

In [14]:
interpreter.init()

[[syntax trees at end of                     typer]] // Main.scala
package <empty> {
  object cmd10$Main extends scala.AnyRef {
    def <init>(): cmd10$Main.type = {
      cmd10$Main.super.<init>();
      ()
    };
    import scala.language.experimental.macros;
    import ReplBridge.shell.{load, publish, evidence, pprintConfig, interpreter};
    import ammonite.api.IvyConstructor.{ArtifactIdExt, GroupIdExt, ResolverNameExt};
    def $main(): Iterator[Iterator[String]] = {
      val $user: cmd10.INSTANCE.$user.type = cmd10.INSTANCE.$user;
      scala.`package`.Iterator.apply[Iterator[String]](scala.`package`.Iterator.apply[String](ReplBridge.shell.shellPPrint[Unit]($user.res10, "res10")((scala.reflect.runtime.`package`.universe.WeakTypeTag.Unit: reflect.runtime.universe.WeakTypeTag[Unit]))).++[String](scala.`package`.Iterator.apply[String](" = ")).++[String](ammonite.pprint.PPrint.apply[Unit]($user.res10)((new ammonite.pprint.PPrint[Unit](ammonite.pprint.PPrinter.Literal, scala.this.Pre

In [15]:
import special.wrap.obj
import scala.language.experimental.macros
import scala.reflect.macros.blackbox.Context

// Allocation-free option type for Scala
// Inspired by https://github.com/arosenberger/nalloc

final class Optional[+A >: Null](val value: A) extends AnyVal {
  def get: A = value
  def isEmpty = value == null
  // @inline final def getOrElse[B >: A](alt: => B): B = if (isEmpty) alt else value
  def getOrElse[B >: A](alt: => B): B = macro OptionalMacros.getOrElse
  override def toString = if (isEmpty) "<empty>" else s"$value"
}

class OptionalMacros(val c: Context) {
  def getOrElse(alt: c.Tree): c.Tree = {
    import c.universe._
    println(showCode(c.macroApplication))
    q"if (isEmpty) $alt else value"
  }
}

import scala.language.experimental.macros
import scala.reflect.macros.blackbox.Context
defined class Optional
defined class OptionalMacros

In [16]:
val x = new Optional(new C)
val x1 = x.getOrElse(new C)
println(x, x1)

val y = new Optional(null)
val y1 = y.getOrElse({ println("hello"); new C })
println(y, y1)

$user.this.x.getOrElse[cmd12.this.$ref$cmd0.C](new cmd12.this.$ref$cmd0.C())
$user.this.y.getOrElse[cmd12.this.$ref$cmd0.C]({
  scala.Predef.println("hello");
  new cmd12.this.$ref$cmd0.C()
})


: 

The above error is expected.

### step 7: let's use quasiquotes to extract the necessary info from c.macroApplication

https://github.com/scalamacros/macrology201/commit/b83f2b7b117d8ae88c0d2d62b8edef71a0bcab9c

In [17]:
import special.wrap.obj
import scala.language.experimental.macros
import scala.reflect.macros.blackbox.Context

// Allocation-free option type for Scala
// Inspired by https://github.com/arosenberger/nalloc

final class Optional[+A >: Null](val value: A) extends AnyVal {
  def get: A = value
  def isEmpty = value == null
  // @inline final def getOrElse[B >: A](alt: => B): B = if (isEmpty) alt else value
  def getOrElse[B >: A](alt: => B): B = macro OptionalMacros.getOrElse
  override def toString = if (isEmpty) "<empty>" else s"$value"
}

class OptionalMacros(val c: Context) {
  def getOrElse(alt: c.Tree): c.Tree = {
    import c.universe._
    val q"$prefix.$_(..$args)" = c.macroApplication
    q"if ($prefix.isEmpty) $alt else $prefix.value"
  }
}

import scala.language.experimental.macros
import scala.reflect.macros.blackbox.Context
defined class Optional
defined class OptionalMacros

In [18]:
val x = new Optional(new C)
val x1 = x.getOrElse(new C)
println(x, x1)

val y = new Optional(null)
val y1 = y.getOrElse({ println("hello"); new C })
println(y, y1)

: 

The above error is expected.

###  step 8: the MatchError hints that we forgot to match type args of the application

https://github.com/scalamacros/macrology201/commit/e41a331faba25ae86074d2f62b6e1800f2853e01

In [19]:
import special.wrap.obj
import scala.language.experimental.macros
import scala.reflect.macros.blackbox.Context

// Allocation-free option type for Scala
// Inspired by https://github.com/arosenberger/nalloc

final class Optional[+A >: Null](val value: A) extends AnyVal {
  def get: A = value
  def isEmpty = value == null
  // @inline final def getOrElse[B >: A](alt: => B): B = if (isEmpty) alt else value
  def getOrElse[B >: A](alt: => B): B = macro OptionalMacros.getOrElse
  override def toString = if (isEmpty) "<empty>" else s"$value"
}

class OptionalMacros(val c: Context) {
  def getOrElse(alt: c.Tree): c.Tree = {
    import c.universe._
    val q"$prefix.$_[..$_](..$args)" = c.macroApplication
    q"if ($prefix.isEmpty) $alt else $prefix.value"
  }
}

import scala.language.experimental.macros
import scala.reflect.macros.blackbox.Context
defined class Optional
defined class OptionalMacros

**FIXME ** jupyter-scala: compiler has to be re-initialized after a macro definition, with a line like the following. Automate this?

In [24]:
interpreter.init()

In [25]:
val x = new Optional(new C)
val x1 = x.getOrElse(new C)
println(x, x1)

val y = new Optional(null)
val y1 = y.getOrElse({ println("hello"); new C })
println(y, y1)

(C,C)
hello
(<empty>,C)


x: specialObjCmd13.Optional[cmd17.INSTANCE.$ref$cmd0.C] = C
x1: cmd17.INSTANCE.$ref$cmd0.C = C
y: specialObjCmd13.Optional[Null] = <empty>
y1: cmd17.INSTANCE.$ref$cmd0.C = C

### step 9: we're not done yet, because we've accidentally changed the flow of control

https://github.com/scalamacros/macrology201/commit/5b35118d5590969cd242aa3125f8f571e71ef156

In [26]:
def x = { println("side-effect!"); new Optional(new C) }
println(x.getOrElse(new C))

side-effect!
side-effect!
C


defined function x

### step 10: introducing a temporary variable fixes the control flow issue 

https://github.com/scalamacros/macrology201/commit/bc0b22ed908bf31bede1bc235c864ac403feb170

In [27]:
import special.wrap.obj
import scala.language.experimental.macros
import scala.reflect.macros.blackbox.Context

// Allocation-free option type for Scala
// Inspired by https://github.com/arosenberger/nalloc

final class Optional[+A >: Null](val value: A) extends AnyVal {
  def get: A = value
  def isEmpty = value == null
  // @inline final def getOrElse[B >: A](alt: => B): B = if (isEmpty) alt else value
  def getOrElse[B >: A](alt: => B): B = macro OptionalMacros.getOrElse
  override def toString = if (isEmpty) "<empty>" else s"$value"
}

class OptionalMacros(val c: Context) {
  def getOrElse(alt: c.Tree): c.Tree = {
    import c.universe._
    val q"$prefix.$_[..$_](..$args)" = c.macroApplication
    q"""
      val temp = $prefix
      if (temp.isEmpty) $alt else temp.value
    """
  }
}

import scala.language.experimental.macros
import scala.reflect.macros.blackbox.Context
defined class Optional
defined class OptionalMacros

In [28]:
interpreter.init() // FIXME jupyter-scala: see comment above

In [29]:
def x = { println("side-effect!"); new Optional(new C) }
println(x.getOrElse(new C))

side-effect!
C


defined function x

### step 11: let's try to hack our macro by inducing a name clash

https://github.com/scalamacros/macrology201/commit/3e8322208a0de705544fde84b521a7900b347b2c

In [30]:
interpreter.init(Seq("-Xprint:typer"))

[[syntax trees at end of                     typer]] // Main.scala
package <empty> {
  object $dummy extends scala.AnyRef {
    def <init>(): $dummy.type = {
      $dummy.super.<init>();
      ()
    }
  }
}



In [31]:
val temp = 100
println(new Optional(if (temp < 100) new C else null).getOrElse(new C))

[[syntax trees at end of                     typer]] // Main.scala
package <empty> {
  object cmd23$Main extends scala.AnyRef {
    def <init>(): cmd23$Main.type = {
      cmd23$Main.super.<init>();
      ()
    };
    private[this] val $ref$cmd0: cmd0.INSTANCE.$user.type = cmd0.INSTANCE.$user;
    <stable> <accessor> def $ref$cmd0: cmd0.INSTANCE.$user.type = cmd23$Main.this.$ref$cmd0;
    import ammonite.api.IvyConstructor.{ArtifactIdExt, GroupIdExt, ResolverNameExt};
    import ReplBridge.shell.{load, publish, evidence, pprintConfig, interpreter};
    import scala.language.experimental.macros;
    import specialObjCmd19.Optional;
    import cmd23$Main.this.$ref$cmd0.C;
    def $main(): Iterator[Iterator[String]] = {
      val $user: cmd23.INSTANCE.$user.type = cmd23.INSTANCE.$user;
      scala.`package`.Iterator.apply[Iterator[String]](scala.`package`.Iterator.apply[String](ReplBridge.shell.shellPPrint[Int]($user.temp, "temp")((scala.reflect.runtime.`package`.universe.WeakTypeTag.Int

temp: Int = 100

### step 12: -uniqid explains the phenomenon we're observing 

https://github.com/scalamacros/macrology201/commit/e779e08a1d753c1997d97981da4fb75960817780

In [32]:
interpreter.init(Seq("-Xprint:typer", "-uniqid"))

[[syntax trees at end of                     typer]] // Main.scala
package <empty> {
  object cmd24$Main extends scala.AnyRef {
    def <init>(): cmd24$Main.type = {
      cmd24$Main.super.<init>();
      ()
    };
    import scala.language.experimental.macros;
    import ReplBridge.shell.{load, publish, evidence, pprintConfig, interpreter};
    import ammonite.api.IvyConstructor.{ArtifactIdExt, GroupIdExt, ResolverNameExt};
    def $main(): Iterator[Iterator[String]] = {
      val $user: cmd24.INSTANCE.$user.type = cmd24.INSTANCE.$user;
      scala.`package`.Iterator.apply[Iterator[String]](scala.`package`.Iterator.apply[String](ReplBridge.shell.shellPPrint[Unit]($user.res24, "res24")((scala.reflect.runtime.`package`.universe.WeakTypeTag.Unit: reflect.runtime.universe.WeakTypeTag[Unit]))).++[String](scala.`package`.Iterator.apply[String](" = ")).++[String](ammonite.pprint.PPrint.apply[Unit]($user.res24)((new ammonite.pprint.PPrint[Unit](ammonite.pprint.PPrinter.Literal, scala.this.Pre

In [33]:
val temp = 100
println(new Optional(if (temp < 100) new C else null).getOrElse(new C))

[[syntax trees at end of                     typer]] // Main.scala
package <empty>#4 {
  object cmd25$Main#28976 extends scala#26.AnyRef#2767 {
    def <init>#28984(): cmd25$Main#28977.type = {
      cmd25$Main#28977.super.<init>#3110();
      ()
    };
    private[this] val $ref$cmd0#28986: cmd0#8702.INSTANCE#28998.$user#29005.type = cmd0#8702.INSTANCE#28998.$user#29005;
    <stable> <accessor> def $ref$cmd0#28985: cmd0#8702.INSTANCE#28998.$user#29005.type = cmd25$Main#28977.this.$ref$cmd0#28986;
    private[this] val $ref$cmd23#28988: cmd23#8354.INSTANCE#29010.$user#29020.type = cmd23#8354.INSTANCE#29010.$user#29020;
    <stable> <accessor> def $ref$cmd23#28987: cmd23#8354.INSTANCE#29010.$user#29020.type = cmd25$Main#28977.this.$ref$cmd23#28988;
    import ammonite#20.api#29024.IvyConstructor#29047.{ArtifactIdExt, GroupIdExt, ResolverNameExt};
    import ReplBridge#8447.shell#29303.{load, publish, evidence, pprintConfig, interpreter};
    import scala#26.language#2527.experimental#29

temp: Int = 100

###  step 13: however it is good practice to give generated variables unique names anyway

https://github.com/scalamacros/macrology201/commit/927535b020bede2bbd5c635f20916f761d7b7d89

In [34]:
// FIXME jupyter-scala: temp was changed to temp0
//                      Having the same name as the temp variable defined above trigger an import of it along
//                      this code, and things go wrong (which ones?).

import special.wrap.obj
import scala.language.experimental.macros
import scala.reflect.macros.blackbox.Context

// Allocation-free option type for Scala
// Inspired by https://github.com/arosenberger/nalloc

final class Optional[+A >: Null](val value: A) extends AnyVal {
  def get: A = value
  def isEmpty = value == null
  // @inline final def getOrElse[B >: A](alt: => B): B = if (isEmpty) alt else value
  def getOrElse[B >: A](alt: => B): B = macro OptionalMacros.getOrElse
  override def toString = if (isEmpty) "<empty>" else s"$value"
}

class OptionalMacros(val c: Context) {
  def getOrElse(alt: c.Tree): c.Tree = {
    import c.universe._
    val q"$prefix.$_[..$_](..$args)" = c.macroApplication
    val temp0 = c.freshName(TermName("temp"))
    q"""
      val $temp0 = $prefix
      if ($temp0.isEmpty) $alt else $temp0.value
    """
  }
}

[[syntax trees at end of                     typer]] // Main.scala
package <empty>#4 {
  import ammonite#20.api#29024.IvyConstructor#29047.{ArtifactIdExt, GroupIdExt, ResolverNameExt};
  import scala#26.reflect#2749.macros#2915.blackbox#29560.Context;
  import ReplBridge#8447.shell#29303.{load, publish, evidence, pprintConfig, interpreter};
  import scala#26.language#2527.experimental#29350.macros;
  import specialObjCmd19#8831.OptionalMacros;
  object specialObjCmd26$Main#43981 extends scala#26.AnyRef#2767 {
    def <init>#43991(): specialObjCmd26$Main#43982.type = {
      specialObjCmd26$Main#43982.super.<init>#3110();
      ()
    };
    def $main#43992(): Iterator#5141[Iterator#5141[String#7795]] = {
      val $user#43995: specialObjCmd26#43983.type = specialObjCmd26#43983;
      scala#26.package#2551.Iterator#3027.apply#29741[Iterator#5141[String#7795]](scala#26.package#2551.Iterator#3027.apply#29741[String#7795](ReplBridge#8447.shell#29303.shellPrintImport#29317("scala.language.e

import scala.language.experimental.macros
import scala.reflect.macros.blackbox.Context
defined class Optional
defined class OptionalMacros

In [35]:
interpreter.init()

[[syntax trees at end of                     typer]] // Main.scala
package <empty>#4 {
  object cmd27$Main#57638 extends scala#26.AnyRef#2767 {
    def <init>#57646(): cmd27$Main#57639.type = {
      cmd27$Main#57639.super.<init>#3110();
      ()
    };
    import scala#26.language#2527.experimental#29350.macros;
    import ReplBridge#8447.shell#29303.{load, publish, evidence, pprintConfig, interpreter};
    import ammonite#20.api#29024.IvyConstructor#29047.{ArtifactIdExt, GroupIdExt, ResolverNameExt};
    def $main#57650(): Iterator#5141[Iterator#5141[String#7795]] = {
      val $user#57653: cmd27#57640.INSTANCE#57655.$user#57662.type = cmd27#57640.INSTANCE#57655.$user#57662;
      scala#26.package#2551.Iterator#3027.apply#29741[Iterator#5141[String#7795]](scala#26.package#2551.Iterator#3027.apply#29741[String#7795](ReplBridge#8447.shell#29303.shellPPrint#29309[Unit#2634]($user#57653.res27#57665, "res27")((scala#26.reflect#2749.runtime#2917.package#15683.universe#15782.WeakTypeTag#121

In [36]:
val temp = 100
println(new Optional(if (temp < 100) new C else null).getOrElse(new C))

C


temp: Int = 100

###  step 14: there's another source for name clash problems that we haven't explored yet

https://github.com/scalamacros/macrology201/commit/e0fff91a10eb74fe1b1c849a2cd477efce4cc5a7

In [37]:
// FIXME jupyter-scala: temp was changed to temp0, and x to x0

import special.wrap.obj
import scala.language.experimental.macros
import scala.reflect.macros.blackbox.Context

// Allocation-free option type for Scala
// Inspired by https://github.com/arosenberger/nalloc

final class Optional[+A >: Null](val value: A) extends AnyVal {
  def get: A = value
  def isEmpty = value == null
  // @inline final def getOrElse[B >: A](alt: => B): B = if (isEmpty) alt else value
  def getOrElse[B >: A](alt: => B): B = macro OptionalMacros.getOrElse
  override def toString = if (isEmpty) "<empty>" else s"$value"
}

object Optional {
  def ensuringNotNull[A](x0: A): A = {
    if (x0 == null) sys.error("argument to Optional.getOrElse can't be null")
    x0
  }
}

class OptionalMacros(val c: Context) {
  def getOrElse(alt: c.Tree): c.Tree = {
    import c.universe._
    val q"$prefix.$_[..$_](..$args)" = c.macroApplication
    val temp0 = c.freshName(TermName("temp"))
    q"""
      val $temp0 = $prefix
      if ($temp0.isEmpty) Optional.ensuringNotNull($alt) else $temp0.value
    """
  }
}

import scala.language.experimental.macros
import scala.reflect.macros.blackbox.Context
defined class Optional
defined object Optional
defined class OptionalMacros

In [38]:
interpreter.init() // FIXME jupyter-scala: see comment above

In [39]:
println(new Optional(null).getOrElse(null))

: 

The above error is expected.

###  step 15: refs to externally defined identifiers typically ask for trouble

https://github.com/scalamacros/macrology201/commit/a2b2743f1e568b233b073b35fc45ab47c9efff1d

In [40]:
val Optional = "uh-oh"
println(new Optional(null).getOrElse(null))

: 

The above error is expected.

### step 16: the only reliable solution here is to use fully-qualified names
### step 17: unfortunately this doesn't always work 

https://github.com/scalamacros/macrology201/commit/da4aacfb058dfdcad94c5dea0cc37ce5907465f2
https://github.com/scalamacros/macrology201/commit/92eb6550f9119a920a6226de9cf35899b4d6b4e2

In [48]:
// FIXME jupyter-scala: temp was changed to temp0, and x to x0
// FIXME jupyter-scala: _root_. ... to reference a just defined type won't work, because the lines are "wrapped"
//                      inside a parent class. How to address that?

import special.wrap.obj
import scala.language.experimental.macros
import scala.reflect.macros.blackbox.Context

// Allocation-free option type for Scala
// Inspired by https://github.com/arosenberger/nalloc

final class Optional[+A >: Null](val value: A) extends AnyVal {
  def get: A = value
  def isEmpty = value == null
  // @inline final def getOrElse[B >: A](alt: => B): B = if (isEmpty) alt else value
  def getOrElse[B >: A](alt: => B): B = macro OptionalMacros.getOrElse
  override def toString = if (isEmpty) "<empty>" else s"$value"
}

object Optional {
  def ensuringNotNull[A](x0: A): A = {
    if (x0 == null) sys.error("argument to Optional.getOrElse can't be null")
    x0
  }
}

class OptionalMacros(val c: Context) {
  def getOrElse(alt: c.Tree): c.Tree = {
    import c.universe._
    val q"$prefix.$_[..$_](..$args)" = c.macroApplication
    val temp0 = c.freshName(TermName("temp"))
    val Optional = q"_root_.Optional" // FIXME jupyter-scala: no the right prefix because of wrapping
    q"""
      val $temp0 = $prefix
      if ($temp0.isEmpty) $Optional.ensuringNotNull($alt) else $temp0.value
    """
  }
}

import scala.language.experimental.macros
import scala.reflect.macros.blackbox.Context
defined class Optional
defined object Optional
defined class OptionalMacros

In [49]:
interpreter.init() // FIXME jupyter-scala: see comment above

In [50]:
val Optional = "uh-oh"
println(new Optional(null).getOrElse(null))

: 

The above error is expected, but the prefix of `Optional` in the macro is wrong because of wrapping in jupyter-scala, so it would fail for other reasons, wasn't for the main error.